In [1]:
%load_ext tensorboard

import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from delta.assets import download_assets
from delta.utilities import cfg
from delta.model import unet_seg
from delta.data import trainGenerator_seg, seg_weights

import pathlib

import datetime


In [2]:
def to_str(posixpath):
    return str(posixpath.resolve())  

In [3]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter')
data_dir = root / 'data'
model_dir = root / 'models'

log_dir = root / 'tf_log' 
log_dir.mkdir(exist_ok=True) 

log_file = to_str(log_dir / 'fit_2022-06-22-16:20')
print(log_file)

new_config_filename = root / 'config_2D_caulobacter.json'

training_set = data_dir / 'training'
validation_set = data_dir / 'validation'

savefile = model_dir / 'unet_caulobacter2d_seg.hdf5'

/scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/tf_log/fit_2022-06-22-16:20


In [6]:
# Load config:
cfg.load_config(new_config_filename)

# Training parameters:
batch_size = 4
epochs = 600
steps_per_epoch = 300
validation_steps = 46 #set such that batch size * validation_steps equals number of validation images
patience = 50

#Data generator parameters:
data_gen_args = dict(
    rotation = 2,
    rotations_90d = True,
    zoom=.15,
    horizontal_flip=True,
    vertical_flip=True,
    histogram_voodoo=True,
    illumination_voodoo=True,
    gaussian_noise = 0.03,
    gaussian_blur = 1
    )

# Generator init:
gen_train = trainGenerator_seg(
    batch_size,
    os.path.join(to_str(training_set),'img'),
    os.path.join(to_str(training_set),'seg'),
    os.path.join(to_str(training_set),'wei'),
    augment_params = data_gen_args,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

gen_validate = trainGenerator_seg(
    batch_size,
    os.path.join(validation_set,'img'),
    os.path.join(validation_set,'seg'),
    os.path.join(validation_set,'wei'),
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

Loading configuration from: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/config_2D_caulobacter.json


In [19]:
# Define model:
model = unet_seg(input_size = cfg.target_size_seg+(1,), pretrained_weights=cfg.model_file_seg)
#model.summary()

In [20]:
#Optional: load weights from previous checkpoint
model.load_weights(to_str(savefile))

In [9]:
a,b = next(gen_validate)

In [10]:
b.shape

(4, 512, 512, 2)

In [21]:
# Callbacks:
model_checkpoint = ModelCheckpoint(
    to_str(savefile), 
    monitor='val_loss', 
    verbose=2, 
    save_best_only=True, 
    save_weights_only=True
    )

early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=2, 
    patience=patience
    )

tensor_board = TensorBoard(log_dir=log_file, histogram_freq=1)

In [16]:
print(log_file)

/scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/tf_log/fit_2022-06-22-16:20


In [ ]:
# Train:
history = model.fit(
    gen_train,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=gen_validate,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint, early_stopping, tensor_board]
    )

Epoch 1/600
300/300 [==============================] - ETA: 0s - loss: 8.2723 - unstack_acc: 0.9925
Epoch 1: val_loss improved from inf to 6.04479, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/unet_caulobacter2d_seg.hdf5
300/300 [==============================] - 223s 739ms/step - loss: 8.2723 - unstack_acc: 0.9925 - val_loss: 6.0448 - val_unstack_acc: 0.9942
Epoch 2/600
300/300 [==============================] - ETA: 0s - loss: 7.4877 - unstack_acc: 0.9929
Epoch 2: val_loss improved from 6.04479 to 5.89992, saving model to /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/models/unet_caulobacter2d_seg.hdf5
300/300 [==============================] - 223s 745ms/step - loss: 7.4877 - unstack_acc: 0.9929 - val_loss: 5.8999 - val_unstack_acc: 0.9947
Epoch 3/600
300/300 [==============================] - ETA: 0s - loss: 7.3746 - unstack_acc: 0.9929
Epoch 3: val_loss improved from 5.89992 to 5.51987, saving model to /scicore/home/vanvli0000/van